# 채팅 형식: The Chat Format

이 노트북에서는 채팅 형식을 활용하여 특정 작업이나 행동에 맞게 개인화되거나 전문화된 챗봇과 확장된 대화를 할 수 있는 방법을 탐구할 것이다.

## 설정

In [ ]:
import os
import openai
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.getenv('OPENAI_API_KEY')

In [ ]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]

def get_completion_from_messages(messages, model="gpt-3.5-turbo", temperature=0):
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature, # this is the degree of randomness of the model's output
    )
#     print(str(response.choices[0].message))
    return response.choices[0].message["content"]

In [ ]:
messages =  [  
{'role':'system', 'content':'당신은 셰익스피어처럼 말하는 assistant입니다.'},    
{'role':'user', 'content':'나에게 농담을 해줘'},   
{'role':'assistant', 'content':'닭은 왜 길을 건넜을까?'},   
{'role':'user', 'content':'나도 몰라요'}  ]

In [ ]:
response = get_completion_from_messages(messages, temperature=1)
print(response)

In [ ]:
messages =  [  
{'role':'system', 'content':'당신은 친절한 챗봇입니다.'},    
{'role':'user', 'content':'안녕, 내 이름은 이사야'}  ]
response = get_completion_from_messages(messages, temperature=1)
print(response)

In [ ]:
messages =  [  
{'role':'system', 'content':'당신은 친절한 챗봇입니다'},    
{'role':'user', 'content':'네, 제 이름이 뭐죠?'}  ]
response = get_completion_from_messages(messages, temperature=1)
print(response)

In [ ]:
messages =  [  
{'role':'system', 'content':'당신은 친절한 챗봇입니다'},
{'role':'user', 'content':'안녕, 내 이름은 이사야'},
{'role':'assistant', 'content': "안녕 이사! 만나서 반가워. \
제가 오늘 도와드릴 일이 있나요?"},
{'role':'user', 'content':'네, 제 이름이 뭐죠?'}  ]
response = get_completion_from_messages(messages, temperature=1)
print(response)

# 주문 봇
우리는 OrderBot을 구축하기 위해 user 프롬프트 및 보조 응답 수집을 자동화할 수 있다. OrderBot은 피자 레스토랑에서 주문을 받을 것이다.

In [ ]:
def collect_messages(_):
    prompt = inp.value_input
    inp.value = ''
    context.append({'role':'user', 'content':f"{prompt}"})
    response = get_completion_from_messages(context) 
    context.append({'role':'assistant', 'content':f"{response}"})
    panels.append(
        pn.Row('user:', pn.pane.Markdown(prompt, width=600)))
    panels.append(
        pn.Row('assistant:', pn.pane.Markdown(response, width=600, style={'background-color': '#F6F6F6'})))
 
    return pn.Column(*panels)


In [ ]:
import panel as pn  # GUI
pn.extension()

panels = [] # collect display 

context = [ {'role':'system', 'content':"""
당신은 피자집의 주문을 받는 자동화된 서비스인 OrderBot입니다.
당신은 먼저 고객에게 인사한 다음 주문을 받은 후 픽업인지 배송인지 물어봅니다.
전체 주문을 수집할 때까지 기다렸다가 요약하고 고객이 추가할 다른 내용이 있는지 최종 확인합니다.
배송이라면, 당신은 주소를 요청합니다.
마침내 당신은 지불금을 회수한다.
메뉴에서 항목을 고유하게 식별할 수 있도록 모든 옵션, 추가 및 크기를 명확히 해야 합니다.
당신은 짧고 매우 대화하기 좋은 스타일로 대답합니다.

메뉴는 다음 항목이 포함됩니다.
페퍼로니 피자 12.95, 10.00, 7.00
치즈피자 10.95, 9.25, 6.50
가지 피자 11.95, 9.75, 6.75
감자튀김 4.50, 3.50
그리스 샐러드 7.25
토핑:
치즈 추가 2.00,
양송이버섯 1.50개
소시지 3.00
캐나다산 베이컨 3.50
AI 소스 1.50
후추 1.00
음료:
코크스 3.00, 2.00, 1.00
스프라이트 3.00, 2.00, 1.00
생수 5.00
"""} ]  # accumulate messages


inp = pn.widgets.TextInput(value="안녕하세요.", placeholder='여기에 텍스트 입력하세요...')
button_conversation = pn.widgets.Button(name="채팅!")

interactive_conversation = pn.bind(collect_messages, button_conversation)

dashboard = pn.Column(
    inp,
    pn.Row(button_conversation),
    pn.panel(interactive_conversation, loading_indicator=True, height=300),
)

dashboard

In [ ]:
messages =  context.copy()
messages.append(
{'role':'system', 'content':'이전 음식 주문의 json 요약을 만듭니다. 각 품목의 가격을 항목화합니다.\
 필드는 1) 피자, 2) 토핑 목록 포함 3) 음료 목록 포함 4) 사이드 목록 포함 5) 총 가격이어야 합니다'},    
)
 # 필드는 1) 피자, 가격 2) 토핑 목록 3) 음료 목록 포함 가격 포함 4) 사이드 목록 포함 가격 포함 가격 포함 가격 포함 5) 총 가격',

response = get_completion_from_messages(messages, temperature=0)
print(response)

## 스스로 실험해 보세요!

메뉴 또는 지시사항을 수정하여 자신만의 주문 봇을 만들 수 있습니다!